<a href="https://colab.research.google.com/github/sanadv/CS360/blob/main/CS360_Dynamic_programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import heapq

def fractional_knapsack_heap(values, weights, capacity):
    n = len(values)

    # Build a max-heap based on ratio (use negative ratios because heapq is a min-heap)
    heap = []
    for i in range(n):
        ratio = values[i] / weights[i]
        # Push (-ratio, value, weight, index)
        heapq.heappush(heap, (-ratio, values[i], weights[i], i))

    total_value = 0.0
    remaining = capacity
    used_items = []   # (item_index, fraction_taken)

    # Pop items in order of highest ratio first
    while heap and remaining > 0:
        neg_ratio, value, weight, index = heapq.heappop(heap)

        if weight <= remaining:
            # take whole item
            total_value += value
            remaining -= weight
            used_items.append((index, 1.0))
        else:
            # take fractional part
            fraction = remaining / weight
            total_value += value * fraction
            used_items.append((index, fraction))
            remaining = 0

    return total_value, used_items


# Example
values  = [60, 100, 120]
weights = [10, 20, 30]
capacity = 50

value, used = fractional_knapsack_heap(values, weights, capacity)
print("Total value:", value)
print("Used items (index, fraction):", used)


Total value: 240.0
Used items (index, fraction): [(0, 1.0), (1, 1.0), (2, 0.6666666666666666)]


In [5]:
def knapsack_01(values, weights, capacity):
    n = len(values)
    best_value = 0
    best_items = []

    # Number of subsets = 2^n
    total_subsets = 1
    for _ in range(n):
        total_subsets *= 2

    # Loop through all subsets represented as numbers (0 to 2^n - 1)
    for subset in range(total_subsets):

        # Convert subset number into a list of booleans (item chosen or not)
        chosen = [0] * n
        x = subset
        for i in range(n):
            chosen[i] = x % 2  # last bit
            x //= 2

        total_value = 0
        total_weight = 0
        items_used = []

        # Calculate weight/value of this subset
        for i in range(n):
            if chosen[i] == 1:   # item included
                total_weight += weights[i]
                total_value += values[i]
                items_used.append(i)

        # Check if valid and best
        if total_weight <= capacity and total_value > best_value:
            best_value = total_value
            best_items = items_used

    return best_value, best_items


# Example:
values  = [60, 100, 120]
weights = [10, 20, 30]
capacity = 50

value, used = knapsack_01(values, weights, capacity)
print("Total value:", value)
print("Items used:", used)


Total value: 220
Items used: [1, 2]


In [2]:
def knapsack_dp(values, weights, capacity):
    n = len(values)

    # Create DP table (n+1) x (capacity+1)
    dp = [[0] * (capacity + 1) for _ in range(n + 1)]

    # Build table
    for i in range(1, n + 1):
        v = values[i - 1]
        w = weights[i - 1]
        for cap in range(capacity + 1):
            if w > cap:
                dp[i][cap] = dp[i - 1][cap]
            else:
                dp[i][cap] = max(dp[i - 1][cap],
                                 dp[i - 1][cap - w] + v)

    # Maximum value
    max_value = dp[n][capacity]

    # ----------------------------------------------------
    # Reconstruct items used
    # ----------------------------------------------------
    used_items = []
    cap = capacity
    i = n

    while i > 0:
        if dp[i][cap] != dp[i - 1][cap]:
            # Item was taken
            used_items.append(i - 1)
            cap -= weights[i - 1]
        i -= 1

    used_items.reverse()  # restore normal order

    return max_value, used_items


# Example usage
values  = [60, 100, 120]
weights = [2, 5, 6]
capacity = 10

maximum, items = knapsack_dp(values, weights, capacity)
print("Maximum value:", maximum)
print("Items used:", items)


Maximum value: 180
Items used: [0, 2]


In [3]:
def lcs_recursive_index(s1: str, s2: str, i: int = 0, j: int = 0):
    # Base case: reached end of either string
    if i == len(s1) or j == len(s2):
        return ""

    # Characters match
    if s1[i] == s2[j]:
        return s1[i] + lcs_recursive_index(s1, s2, i + 1, j + 1)

    # Characters do not match
    option1 = lcs_recursive_index(s1, s2, i + 1, j)
    option2 = lcs_recursive_index(s1, s2, i, j + 1)

    return option1 if len(option1) >= len(option2) else option2
a = "AGGTAB"
b = "GXTXAYB"
result = lcs_recursive_index(a, b)
print("LCS:", result)
print("Length:", len(result))

LCS: GTAB
Length: 4


In [16]:
def longest_common_substring_iterative(s1, s2):
    best_len = 0
    best_i = 0
    best_j = 0

    for i in range(len(s1)):
        for j in range(len(s2)):

            # match_from loop
            length = 0
            ii, jj = i, j

            while ii < len(s1) and jj < len(s2) and s1[ii] == s2[jj]:
                length += 1
                ii += 1
                jj += 1

            # update best
            if length > best_len:
                best_len = length
                best_i = i
                best_j = j

    return s1[best_i : best_i + best_len]


# Test
a = "abcdffgab"
b = "xycdffab"
print(longest_common_substring_iterative(a, b))  # "cdff"


cdff


In [17]:
def longest_common_substring_dp(s1, s2):
    n, m = len(s1), len(s2)
    if n == 0 or m == 0:
        return ""

    # dp[i][j] = length of longest common substring ending at s1[i-1], s2[j-1]
    dp = [[0] * (m + 1) for _ in range(n + 1)]

    best_len = 0
    best_i = 0  # start index in s1

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if s1[i - 1] == s2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1

                if dp[i][j] > best_len:
                    best_len = dp[i][j]
                    best_i = i - best_len
            else:
                dp[i][j] = 0  # substrings must be contiguous

    return s1[best_i : best_i + best_len]


# Test
a = "abcdffgab"
b = "xycdffab"
print(longest_common_substring_dp(a, b))  # "cdff"


cdff
